##### Copyright 2019 The TensorFlow Authors.

In [14]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初心者のための TensorFlow 2.0 入門

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs-l10n](https://github.com/tensorflow/docs-l10n)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

この短いイントロダクションでは [Keras](https://www.tensorflow.org/guide/keras/overview) を使って下記のことを行います。

1. 画像を分類するニューラルネットワークを構築する
2. このニューラルネットワークを訓練する
3. そして最後に、モデルの正解率を評価する

このファイルは [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) の notebook ファイルです。 Python プログラムはブラウザ上で直接実行されます。TensorFlow を学んだり使ったりするには最良の方法です。Google Colab のnotebook の実行方法は以下のとおりです。

1. Pythonランタイムへの接続：メニューバーの右上で「接続」を選択します。
2. ノートブックのコードセルをすべて実行：「ランタイム」メニューから「すべてのセルを実行」を選択します。

TensorFlow 2 のパッケージをダウンロードしてインストールします。プログラムに TensorFlow をインポートします。

Note: TensorFlow 2 パッケージをインストールするためには `pip` をアップグレードしてください。詳細は [インストール](https://www.tensorflow.org/install)を参照ください。

In [15]:
import tensorflow as tf

[MNIST データセット](http://yann.lecun.com/exdb/mnist/)をロードして準備します。サンプルを整数から浮動小数点数に変換します。

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

層を積み重ねて`tf.keras.Sequential`モデルを構築します。訓練のためにオプティマイザと損失関数を選びます。

In [17]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

モデルはそれぞれの標本について、クラスごとに"[ロジット](https://developers.google.com/machine-learning/glossary#logits)"や"[対数オッズ比](https://developers.google.com/machine-learning/glossary#log-odds)"と呼ばれるスコアを算出します。

In [18]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.12310395, -0.1660875 ,  0.19069259, -0.30691612, -1.0037296 ,
        -0.17386752,  0.7962265 , -0.36067355,  0.7561678 , -0.02146456]],
      dtype=float32)

`tf.nn.softmax` 関数はクラスごとにこれらのロジットを "確率" に変換します。

In [19]:
tf.nn.softmax(predictions).numpy()

array([[0.08106396, 0.07765336, 0.11094505, 0.06745268, 0.03360292,
        0.07705157, 0.20327686, 0.06392233, 0.1952948 , 0.08973652]],
      dtype=float32)

Note: `tf.nn.softmax` をネットワークの最終レイヤーに対する活性化関数として組み込むことは可能です。これはモデルの出力に対して直接的に解釈できるようにする一方で、ソフトマックスの出力を用いる場合、すべてのモデルに対して正確で数値的安定に損失を計算することが不可能なため、このアプローチは推奨されません。

`losses.SparseCategoricalCrossentropy` 損失関数は、ロジットと `True` の インデックスに関するベクトルを入力にとり、それぞれの標本についてクラスごとに損失のスカラーを返します。

In [20]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

この損失は、クラスが正しい確率の対数をとって符号を反転させたものです。この値はモデルがこのクラスが正しいと確信しているときに 0 になります。

この訓練されていないモデルはランダムに近い確率 (それぞれのクラスについて 1/10) を出力します、最初の損失は `-tf.log(1/10) ~= 2.3` に近い値になるはずです。

In [21]:
loss_fn(y_train[:1], predictions).numpy()

2.5632803

In [22]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

`Model.fit` メソッドは損失を最小化するようにモデルのパラメータを調整します。

In [23]:
import time
start = time.time()
model.fit(x_train, y_train, epochs=5)
print(f"Time is {time.time()-start}s")

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2948 - accuracy: 0.9143
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1477 - accuracy: 0.9569
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1097 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0905 - accuracy: 0.9721
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0774 - accuracy: 0.9757
Time is 15.302536964416504s


`Model.evaluate` メソッドはモデルの性能を検査します。これには通常 "[検証用データセット](https://developers.google.com/machine-learning/glossary#validation-set)" または "[テストデータセット](https://developers.google.com/machine-learning/glossary#test-set)" を用います。

In [24]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0842 - accuracy: 0.9750


[0.08418061584234238, 0.9750000238418579]

この画像識別器はこのデータセットにおいておよそ 98% の正解率にまで訓練されました。更に学ぶためには [TensorFlow tutorials](https://www.tensorflow.org/tutorials/) を参照ください。

モデルが確率を返すようにしたい場合には、モデルをラップしてソフトマックス関数を適用してください。

In [25]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [26]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.9535440e-08, 5.9966870e-10, 1.7643022e-05, 4.3439286e-04,
        2.0001197e-11, 5.7169871e-07, 1.7831820e-12, 9.9952912e-01,
        1.9532815e-06, 1.6219168e-05],
       [1.7942182e-07, 1.4664479e-04, 9.9985135e-01, 6.6766745e-07,
        4.8884511e-16, 5.1744344e-08, 9.4953016e-08, 7.8505878e-16,
        9.3188203e-07, 3.7556051e-16],
       [3.6617536e-07, 9.9927050e-01, 1.2045458e-04, 5.0078152e-06,
        5.7429443e-06, 1.9653610e-06, 9.0218155e-06, 5.2132219e-04,
        6.5133245e-05, 5.8782013e-07],
       [9.9866641e-01, 1.9851832e-08, 7.0672814e-04, 1.0643579e-06,
        1.5446118e-05, 4.3724300e-05, 4.8139939e-04, 4.9172602e-05,
        3.0184600e-07, 3.5720986e-05],
       [1.0706171e-05, 4.8879514e-08, 1.3617876e-04, 4.9704659e-07,
        9.9325800e-01, 1.2935894e-06, 1.8765143e-04, 1.1831292e-04,
        6.3152625e-06, 6.2809996e-03]], dtype=float32)>